# Select Your IPTS 

In [14]:
from __code.normalization_new_ui import *

from __code.ui_builder import UiBuilder
o_builder = UiBuilder(ui_name = 'ui_roi_selection.ui')
from __code.roi_selection_ui import Interface

from __code import system
system.System.select_working_dir()
from __code.__all import custom_style
custom_style.style()

pyuic5 /Users/j35/git/IPTS/python_notebooks/ui/ui_roi_selection.ui -o /Users/j35/git/IPTS/python_notebooks/__code/ui_roi_selection.py
** Using Debugging Mode! **


HBox(children=(Label(value='Select Working Folder', layout=Layout(width='20%')), Select(index=9, options=('BraggEdge', 'Brianne', 'DOE_letters', 'FITSimages', 'IPTS-12534', 'IPTS-20209-TOPAZ', 'IPTS_13639_resonance_shawn', 'IPTS_14832', 'IPTS_16212', 'IPTS_16259', 'IPTS_16475', 'IPTS_16893', 'IPTS_17099', 'IPTS_17685', 'IPTS_17981', 'IPTS_18580', 'IPTS_18806', 'IPTS_18813', 'IPTS_19037', 'IPTS_19558', 'IPTS_19621', 'IPTS_19626', 'IPTS_test_normalization', 'IPTS_testing_time_spectra', 'ImagesToRegisterForGian', 'Indu', 'Keita_code', 'Pour_ibeatles_video', 'ResonanceImaging', 'SENSITIVE', 'SarmaGUI', 'SphereCoordinates3D', 'TempData', 'gian', 'iBeatles_debugging', 'shared'), value='IPTS_16259')))

# Description 

This notebook performs the normalization of a set of images (tiff or fits) by OBs. 

The steps are:
 - select your **sample images**
 - select your **ob images**
 - select your **df images** (optional)
 - all the data will be loaded
 - preview of the sample and ob
 - optionally, selection of one or more **background** region in the sample data
 - normalization is performed
 - export of the normalized data

# Python Import

In [15]:
%gui qt

In [16]:
%matplotlib notebook

# Loading Images 

In [17]:
files = Files()
sample_panel = SampleSelectionPanel(working_dir=system.System.get_working_dir())
sample_panel.init_ui(files=files)
wizard = WizardPanel(sample_panel=sample_panel)

HTML(value='Please wait...')

Label(value='Selection of All Input Files', layout=Layout(border='1px lighgray solide', height='35px', padding='8px', width='300px'))

VBox(children=(HBox(children=(Label(value='Instructions:', layout=Layout(width='20%')), Label(value='Select list of Sample files (mandatory) ', layout=Layout(width='50%')))), HBox(children=(Label(value='Sample selected:', layout=Layout(width='20%')), Label(value='0 files selected', layout=Layout(width='50%'))))), layout=Layout(border='1px lightgray solid', margin='5px', padding='15px'))

HBox(children=(Label(value='         ', layout=Layout(width='70%')), Button(button_style='warning', description='Next Step>>', disabled=True, layout=Layout(width='20%'), style=ButtonStyle(), tooltip='Click to move to next step')))

VBox(children=(Label(value='', layout=Layout(width='250px')), HBox(children=(HBox(children=(Text(value='/Volumes/my_book_thunderbolt_duo/IPTS/IPTS_18813', description='Location: ', layout=Layout(width='300px'), placeholder=''), Button(description='Jump', layout=Layout(margin='5px 10px', width='100px'), style=ButtonStyle())), layout=Layout(border='1px solid lightgrey', margin='5px 50px 5px 5px', padding='3px')),)), VBox(children=(SelectMultiple(description='Select', layout=Layout(display='flex', flex_flow='column', width='750px'), options=(' .', ' ..', ' .DS_Store                              |     Tue Jan 30 16:04:04 2018', ' DF                                     |     Directory', ' OB                                     |     Directory', ' Sample2_Ethane                         |     Directory', ' Sample3_Methane                        |     Directory', ' Sample_2                               |     Directory', ' Sample_2_light                         |     Directory', ' Sample_2_light_timestamp_infos.txt     |     Wed Jan 31 11:57:35 2018'), value=()), HBox(children=(Button(description='Enter directory', layout=Layout(margin='5px 40px'), style=ButtonStyle()), Button(description='Select', layout=Layout(margin='5px 40px'), style=ButtonStyle())))), layout=Layout(border='1px solid lightgrey', margin='5px', padding='10px'))))

...

HTML(value='Please wait...')

VBox(children=(Label(value='', layout=Layout(width='250px')), HBox(children=(HBox(children=(Text(value='/Volumes/my_book_thunderbolt_duo/IPTS/IPTS_18813/DF', description='Location: ', layout=Layout(width='300px'), placeholder=''), Button(description='Jump', layout=Layout(margin='5px 10px', width='100px'), style=ButtonStyle())), layout=Layout(border='1px solid lightgrey', margin='5px 50px 5px 5px', padding='3px')),)), VBox(children=(SelectMultiple(description='Select', layout=Layout(display='flex', flex_flow='column', width='750px'), options=(' .', ' ..', ' 20180117_DF_0030_0913.tiff     |     Wed Jan 17 09:12:11 2018', ' 20180117_DF_0030_0914.tiff     |     Wed Jan 17 09:12:46 2018', ' 20180117_DF_0030_0915.tiff     |     Wed Jan 17 09:13:21 2018', ' 20180117_DF_0030_0916.tiff     |     Wed Jan 17 09:13:56 2018', ' 20180117_DF_0030_0917.tiff     |     Wed Jan 17 09:14:31 2018', ' 20180117_DF_0030_0918.tiff     |     Wed Jan 17 09:15:06 2018', ' 20180117_DF_0030_0919.tiff     |     Wed Jan 17 09:15:41 2018', ' 20180117_DF_0030_0920.tiff     |     Wed Jan 17 09:16:15 2018', ' 20180117_DF_0030_0921.tiff     |     Wed Jan 17 09:16:50 2018', ' 20180117_DF_0030_0922.tiff     |     Wed Jan 17 09:17:25 2018'), value=()), HBox(children=(Button(description='Enter directory', layout=Layout(margin='5px 40px'), style=ButtonStyle()), Button(description='Select', layout=Layout(margin='5px 40px'), style=ButtonStyle())))), layout=Layout(border='1px solid lightgrey', margin='5px', padding='10px'))))

HBox(children=(Label(value='Loading sample', layout=Layout(width='20%')), IntProgress(value=0, max=5)))

HBox(children=(Label(value='Loading ob', layout=Layout(width='20%')), IntProgress(value=0, max=5)))

HBox(children=(Label(value='Loading df', layout=Layout(width='20%')), IntProgress(value=0, max=2)))

# Preview Data

## sample

In [18]:
o_norm = sample_panel.o_norm
o_norm.plot_images(data_type='sample')

interactive(children=(IntSlider(value=0, continuous_update=False, description='sample Index', max=4), Output()), _dom_classes=('widget-interact',))

## ob

In [19]:
o_norm.plot_images(data_type='ob')

interactive(children=(IntSlider(value=0, continuous_update=False, description='ob Index', max=4), Output()), _dom_classes=('widget-interact',))

# Select Background Region 

In [20]:
o_gui = Interface(o_norm=o_norm.o_norm)
o_gui.show()

Leaving Parameters Selection UI


# Recap. 

In [21]:
o_gui.recap()

Type,Number,Size (height*width)
sample,5,2048*2048
ob,5,2048*2048
df,2,2048*2048


# Normalization

In [22]:
o_norm.run_normalization(dict_roi=o_gui.roi_selected)

HBox(children=(Label(value='Normalization', layout=Layout(width='20%')), IntProgress(value=0, max=5)))

# Export 

In [23]:
o_norm.select_export_folder()

HTML(value='Please wait...')

VBox(children=(Label(value='Select Output Folder', layout=Layout(width='250px')), HBox(children=(HBox(children=(Text(value='/Volumes/my_book_thunderbolt_duo/IPTS/IPTS_18813', description='Location: ', layout=Layout(width='300px'), placeholder=''), Button(description='Jump', layout=Layout(margin='5px 10px', width='100px'), style=ButtonStyle())), layout=Layout(border='1px solid lightgrey', margin='5px 50px 5px 5px', padding='3px')),)), VBox(children=(Select(description='Select', layout=Layout(width='750px'), options=(' .', ' ..', ' .DS_Store                              |     Tue Jan 30 16:04:04 2018', ' DF                                     |     Directory', ' OB                                     |     Directory', ' Sample2_Ethane                         |     Directory', ' Sample3_Methane                        |     Directory', ' Sample_2                               |     Directory', ' Sample_2_light                         |     Directory', ' Sample_2_light_timestamp_infos.txt     |     Wed Jan 31 11:57:35 2018'), value=' .'), HBox(children=(Button(description='Enter directory', layout=Layout(margin='5px 40px'), style=ButtonStyle()), Button(description='Select', layout=Layout(margin='5px 40px'), style=ButtonStyle())))), layout=Layout(border='1px solid lightgrey', margin='5px', padding='10px'))))

In [24]:
o_norm.export()

IntProgress(value=0, max=5)